In [40]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import timeit
from IPython.display import display
from tqdm import tqdm_notebook
from sklearn.decomposition import NMF

In [ ]:
# load the data
df1 = pd.read_csv('item_properties_part1.csv')
df2 = pd.read_csv('item_properties_part2.csv')

properties_df = pd.concat([df1, df2])
events_df = pd.read_csv('events.csv')
category_df = pd.read_csv('category_tree.csv')

# change timestamp to more useful format
properties_df.timestamp = pd.to_datetime(properties_df['timestamp'], unit='ms')
events_df.timestamp = pd.to_datetime(events_df['timestamp'], unit='ms')

In [ ]:
# first look at data
display(properties_df.head())
display(events_df.head())
display(category_df.head())

In [ ]:
# We have plenty of rows in properties and only "property" values of "categoryid" and "available" are not hashed
# Thus we decide to remove all rows except where "availalbe" is 1 or we have the "categoryid"
# df_filtered = df[df['Age'] >= 25]
properties_df = properties_df[(properties_df.property == "categoryid") | ((properties_df.property == "available") & (properties_df.value == '1'))]

In [ ]:
properties_df.property.value_counts()

In [ ]:
properties_df.head()

In [ ]:
# Data exploration
# check for missing values and look at histograms and counts

# how many na values in category parent and child columns - there are 25 categories with no parent
print('na category values', category_df.isnull().sum(axis = 0))

# number of rows in category df
print('number of rows in category df', category_df.shape[0])

# how many unique parent category ids
print('unique parent', category_df['parentid'].unique().size)

# how many unique child category ids
print('unique child', category_df['categoryid'].unique().size)

# how many children ids to the top parent categories have
n = 20
counts = pd.value_counts(category_df['parentid'].values)
print("child counts for top {} parent IDs".format(n), sorted(counts, reverse=True)[:n])

# clearly events are dominated by views and there are no missing values in the events column
counts = pd.value_counts(events_df['event'].values)
print(counts)
counts.plot.bar(rot=0)

In [ ]:
# since there are only 25 rows with nan values in category_df we decide to eliminate them
category_df = category_df.dropna()

In [ ]:
# we have too many items at the moment - causing too many columns in our ranking matrix
# we determine to remove items that have been viewed fewer than 30 times by all visitors and that have not been added to a cart or purchased
'''
This function takes in the events data frame and minimum thresholds for the "event"'s, "view", "addtocart", 
and "transaction" and returns a data frame that only includes "itemid"'s, products, that have been viewed, added, or purchased
enough times to meet one of the minimum threholds. Basically, we are only looking at items that meet a minimum activity requiremnt.
Since we have to cut some items for both memory and compuational efficiency it made the most sense keep "hot" itmes for which rankings
would be the most valuable.
'''
def get_reduced_events(e_df=events_df, view_threshold =  200, add_to_cart_threshold = 40, transaction_threshold = 20):
    view_counts = pd.value_counts(events_df[events_df.event == 'view']['itemid'].values)
    add_to_cart_counts = pd.value_counts(events_df[events_df.event == 'addtocart']['itemid'].values)
    transaction_counts = pd.value_counts(events_df[events_df.event == 'transaction']['itemid'].values)
    keep_view = view_counts[view_counts.values >= view_threshold]
    keep_add_to_cart = add_to_cart_counts[add_to_cart_counts.values >= add_to_cart_threshold]
    keep_transaction = transaction_counts[transaction_counts.values >= transaction_threshold]
    events_reduced_df = events_df[(events_df.itemid.isin(keep_view.index)) | (events_df.itemid.isin(keep_add_to_cart.index)) | (events_df.itemid.isin(keep_transaction.index))]
    return events_reduced_df

In [ ]:
'''
This fuction takes in the events, properties, and category data frames.
It also takes two boolean parameters for addtocart and transaction events
when these values are true it keeps track of itemids that are either added to cart or purchased in each row
example: if both parameters are set to True and in row 1 items 1105 and 1207 were added to cart and or purchased 
the target value, y_array, whould have the set {1105, 1207} stored as the ground truth for row 1.
Additionally it takes weights for the three event types for a given item, its siblings (items with a common categoryid),
and its cousins (itmes with a common parentid (some categoryids have parentids)).
Finally it has two additional boolean parameters, realtive and distant relative, that turn the checks for siblings and cousins
on and off.
It returns the rank matrix, y_vals and itemids
'''
def get_rank_matrix(e_df=events_df, p_df=properties_df, c_df=category_df, 
                    y_event_add_to_cart=True, y_event_transaction=True,
                    item_view=3, item_add=10, item_purchase=10, 
                    relative_view=2, relative_add=5, relative_purchase=5, relative=False, 
                    distant_relative_view=1, distant_relative_add=2, distant_relative_purchase=2, distant_relative=False,
                   ):
    
    products = np.intersect1d(e_df.itemid.unique(), p_df.itemid.unique(), assume_unique=True)
    e_df = e_df[e_df.itemid.isin(products)]
    p_df = p_df[p_df.itemid.isin(products)]
    
    visitors = e_df.visitorid.unique()
    
    rank_matrix = np.zeros((visitors.size, products.size))
    
    item_idx_dict = dict(zip(products,range(products.size)))

    y_vals = [set() for x in range(visitors.size)]
    y_events = np.array(['addtocart', 'transaction'])
    y_events = y_events[[y_event_add_to_cart, y_event_transaction]]
    
    print('rank_matrix_size is', rank_matrix.shape)

    events_dict = defaultdict(lambda: defaultdict(lambda: []))
    for event, visitorid, itemid in e_df[['event', 'visitorid', 'itemid']].values:
        events_dict[event][visitorid].append(itemid)
    
    if relative or distant_relative:
        common_df = p_df[(p_df.property == 'categoryid')]
        common_df.value = common_df.value.astype(int)
        
        relatives_dict = defaultdict(lambda: set())
        for value, itemid in common_df[['value', 'itemid']].values:
            relatives_dict[value].add(itemid)
            
        common_dict = common_df.set_index('itemid').to_dict()['value']
        
        siblings_dict = {}
        for itemid, categoryid in common_dict.items():
            if categoryid in relatives_dict:
                siblings_dict[itemid] = relatives_dict[categoryid] - set([itemid])
                
        if distant_relative:
            category_ids = common_df.value.unique()
            c_df = c_df[c_df.categoryid.isin(category_ids)]

            parent_dict = defaultdict(lambda: set())
            for parentid, categoryid in c_df[['parentid', 'categoryid']].values:
                parent_dict[parentid].add(categoryid)

            union_dict = c_df.set_index('categoryid').to_dict()['parentid']

            categoryid_siblings_dict = {}
            for categoryid, parentid in union_dict.items():
                if parentid in parent_dict:
                    categoryid_siblings_dict[categoryid] = parent_dict[parentid] - set([categoryid])

            cousins_dict = {}
            for itemid, sibling_set in siblings_dict.items():
                temp_set = set()
                if common_dict[itemid] in categoryid_siblings_dict:
                    for uncle in categoryid_siblings_dict[common_dict[itemid]]:
                        temp_set.union(relatives_dict[uncle])
                cousins_dict[itemid] = temp_set - sibling_set
    
    events = ['view', 'addtocart', 'transaction']
    item_weights = {'view':item_view, 'addtocart':item_add, 'transaction':item_purchase}
    relative_weights = {'view':relative_view, 'addtocart':relative_add, 'transaction':relative_purchase}
    distant_relative_weights = {'view':relative_view, 'addtocart':relative_add, 'transaction':relative_purchase}
    
    for row, visitor in tqdm_notebook(enumerate(visitors), total = len(visitors)):
        for col, product in enumerate(products):
            for event in events:
                if visitor in events_dict[event] and product in events_dict[event][visitor]:
                    rank_matrix[row,col] += item_weights[event]
                    if event in y_events:
                        y_vals[row].add(product)
                    if relative and product in siblings_dict:
                        for sibling in siblings_dict[product]:
                            rank_matrix[row, item_idx_dict[sibling]] += relative_weights[event]
                    if distant_relative and product in cousins_dict:
                        for cousin in cousins_dict[product]:
                            rank_matrix[row, item_idx_dict[cousin]] += distant_relative_weights[event]
        
    return rank_matrix, np.fromiter(item_idx_dict.keys(), dtype=int), y_vals

In [ ]:
'''
This function takes in the rank matrix, the numpy array of itemid's used to create the rank matrix
the maximum number of predicted items you want returned for each row and a minimum threhold for rank value.
The minimum threhold must be met for any item to be predicted. If it is not met an empty set is returned for that row
regardless of the number of predicted items selected.
'''
def get_y_predict(rank_matrix, item_ids, num_top_vals=2, event_threshold=5):
    index_array = np.argpartition(rank_matrix, num_top_vals)[:,-num_top_vals:]
    y_predict = [set() for x in range(rank_matrix.shape[0])]
    for row, indices in enumerate(index_array):
        for idx in indices:
            if rank_matrix[row,idx] > event_threshold:
                y_predict[row].add(item_ids[idx])
    return y_predict

In [ ]:
'''
This function takes in the predicted items and compares them to the 
actual addtocart and or transaction items (depending on what was selected)
it returns both a total hit rate ((correct_empty_predictions + correct_positive_predictions)/prediction_count)
and a positive hit rate (correct_positive_predictions/number_of_times_ground_truth_was_not_empty)
Note: a correct positive prediction occurs anytime the intersection between the predection set 
and groundtruth set is not empty thus correctly guessing one of the items in the ground truth set results
in being counted as a correct prediction.
'''
def get_hit_rate(y_vals, y_predict):
    hits = 0
    both_empty = 0
    y_not_empty = 0
    for idx in range(len(y_predict)):
        if (not y_vals[idx] and not y_predict[idx]):
            both_empty += 1
        elif y_vals[idx]:
            y_not_empty += 1
            if y_vals[idx].intersection(y_predict[idx]):
                hits += 1
    total_rate = (both_empty + hits)/len(y_predict)
    positive_hit_rate = hits/y_not_empty
    return total_rate, positive_hit_rate

In [ ]:
# rank_matrix = get_rank_matrix(e_df=events_df.sample(n=int(.005*events_df.shape[0]), random_state=881))
rank_matrix, itemids, y_vals = get_rank_matrix(e_df=get_reduced_events(), relative=True, distant_relative=True, relative_add=0, relative_purchase=0, item_add=0, item_purchase=0, )

In [ ]:
y_predict = get_y_predict(rank_matrix, itemids, num_top_vals=2, event_threshold=20)

In [ ]:
total_accuracy, hit_rate = get_hit_rate(y_vals, y_predict)

In [ ]:
print(total_accuracy, hit_rate)

In [ ]:
def create_recommender(rank_matrix, n_components = 20):
    model = NMF(init='nndsvdar', verbose=1, n_components=20)
    W = model.fit_transform(rank_matrix)
    H = model.components_
    return np.dot(W,H)

In [34]:
def generate_recommendations(recommender, num_recs = 10):
    y_predict = [set() for x in range(rank_matrix.shape[0])]
    for index,row in tqdm_notebook(enumerate(recommender), total=len(recommender)):
        sorted_row = sorted(row)
        y_predict[index].update(set(sorted_row[-num_recs:]))
    return y_predict

In [ ]:
recommender = create_recommender(rank_matrix)

In [41]:
recs = generate_recommendations(recommender)

Exception ignored in: <function tqdm.__del__ at 0x0000027DCDEE5040>
Traceback (most recent call last):
  File "C:\Program Files\Python38\lib\site-packages\tqdm\std.py", line 1124, in __del__
    self.close()
  File "C:\Program Files\Python38\lib\site-packages\tqdm\notebook.py", line 271, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <function tqdm.__del__ at 0x0000027DCDEE5040>
Traceback (most recent call last):
  File "C:\Program Files\Python38\lib\site-packages\tqdm\std.py", line 1124, in __del__
    self.close()
  File "C:\Program Files\Python38\lib\site-packages\tqdm\notebook.py", line 271, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html